# Monitoramento de Rotas e Latência com TraceRoute e Ping
## Redes de Computadores
### Alunos: Leonardo de Oliveira Nanes, Gabriel Lázaro e Gabriel Calabrese 

Nesta apresentação, vamos explorar o conceito de **TraceRoute** com **Ping** imbutido para monitoramento de redes e visualização de hops, latência e perda de pacotes entre o host e o destino. Além disso, plotaremos as rotas geograficamente em um mapa interativo.

---

### Estrutura da apresentação:
-  Introdução aos conceitos de TraceRoute e Ping.
-  Implementação prática das ferramentas.
-  Visualização das rotas no mapa.

---

## O que é TraceRoute?

O **TraceRoute** é uma ferramenta de diagnóstico de redes que rastreia o caminho percorrido pelos pacotes de dados entre a origem (host) e o destino. Ele identifica os roteadores intermediários (hops) que os pacotes atravessam até chegar ao seu destino final.

- **Funcionamento**: O TraceRoute envia pacotes com valores incrementais de TTL (Time to Live). Quando o TTL chega a 0, o roteador emite uma mensagem ICMP "time exceeded", revelando sua presença. Esse processo se repete até alcançar o destino final ou até o TTL máximo.

---

## O que é Ping?

O **Ping** é uma ferramenta usada para testar a conectividade entre dois dispositivos em uma rede. Ele mede a latência (tempo de resposta) e a perda de pacotes. O Ping utiliza pacotes ICMP "Echo Request" e "Echo Reply" para testar se um dispositivo está acessível e para obter informações sobre o tempo de resposta.

- **Latência**: Tempo que um pacote leva para ir até o destino e voltar.
- **Perda de Pacotes**: Proporção de pacotes enviados que não receberam resposta.


# IMPORTAÇÃO DE BIBLIOTECAS

In [37]:
import socket
import folium
import requests
import ipaddress
import pingparsing


## Funções: TraceRoute, Ping e Geolocalização

Aqui estão as funções principais para realizar o TraceRoute com Ping e para obter as coordenadas dos IPs através de uma API pública.

- `traceroute_with_ping`: Realiza o TraceRoute até o destino especificado, verificando a latência e perda de pacotes a cada hop.
- `get_coordinates`: Obtém as coordenadas geográficas do IP usando uma API para plotagem de roteadores no mapa.
- `is_private_ip`: Verifica se o IP é privado e, assim, não acessível por APIs públicas.


# TRACEROUTE COM PING

In [38]:


def traceroute_with_ping(host, max_hops=30, timeout=2, ping_count=4):
    print(f"TraceRoute para {host}:")

    hops = []
    dest_addr = socket.gethostbyname(host)
    port = 33434  
    icmp = socket.getprotobyname('icmp')
    udp = socket.getprotobyname('udp')
    ping_parser = pingparsing.PingParsing()
    transmitter = pingparsing.PingTransmitter()

    for ttl in range(1, max_hops + 1):
        recv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, icmp)
        send_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, udp)
        send_socket.setsockopt(socket.SOL_IP, socket.IP_TTL, ttl)
        recv_socket.settimeout(timeout)

        recv_socket.bind(("", port))
        send_socket.sendto(b"", (host, port))

        curr_addr = None
        try:
            _, curr_addr = recv_socket.recvfrom(512)
            curr_addr = curr_addr[0]
            hops.append(curr_addr)

            # Fazer ping no hop atual
            transmitter.destination = curr_addr
            transmitter.count = ping_count
            ping_result = transmitter.ping()

            if ping_result.returncode == 0:
                ping_stats = ping_parser.parse(ping_result.stdout)

                packet_loss = ping_stats.packet_loss_rate
                packets_sent = ping_stats.packet_transmit
                min_time = ping_stats.rtt_min
                avg_time = ping_stats.rtt_avg
                max_time = ping_stats.rtt_max
                mdev_time = ping_stats.rtt_mdev

                print(
                    f"{ttl}: {curr_addr} | Sent: {packets_sent} | Loss: {packet_loss} % | avg: {avg_time}ms | Best: {min_time}ms | Worst: {max_time}ms | Std.Dev: {mdev_time}")
            else:
                print(f"{ttl}: {curr_addr} | Erro no ping")

        except socket.timeout:
            print(f"{ttl}: * (timeout)")
        except socket.error as e:
            print(f"{ttl}: erro no socket - {e}")
        finally:
            send_socket.close()
            recv_socket.close()

        if curr_addr == dest_addr:
            print("Destino alcançado!")
            break
    return hops

# MAPA

## Vendo se o IP é privado

In [39]:
def is_private_ip(ip):
    return ipaddress.ip_address(ip).is_private

## Pegando as coordenadas

In [40]:
def get_coordinates(ip):
    if is_private_ip(ip):
        print(f"IP {ip} é privado. Sem coordenadas públicas conhecidas.")
        return None

    try:
        response = requests.get(f"http://ip-api.com/json/{ip}")
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'success':
                lat, lon = data['lat'], data['lon']
                # Exibe as coordenadas no console
                print(f"Coordenadas de {ip}: ({lat}, {lon})")
                return lat, lon
            else:
                print(f"API retornou falha para o IP {ip}: {data['message']}")
                return None
        else:
            print(
                f"Erro ao conectar-se à API para o IP {ip}. Status HTTP: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Erro de conexão ao tentar obter localização de {ip}: {e}")
        return None

# PLOTANDO O MAPA

In [41]:
from folium.plugins import TimestampedGeoJson
from datetime import datetime,timedelta


def plot_route(hops):
    linhas = []
    data_inicial = datetime(2024, 1, 1, 0, 0, 0)  # Data inicial arbitrária
    intervalo_tempo = timedelta(minutes=1)  # A cada salto, adicionar 1 minuto (ou o intervalo que desejar)

    def inverter_coordenada(coordenada):
        latitude, longitude = coordenada
        return longitude, latitude

    for i, hop in enumerate(hops):
        coords = get_coordinates(hop)
        if coords:  # Verifica se coords não é None
            coords = inverter_coordenada(coords)
            # Incrementa o tempo para cada salto
            data_atual = data_inicial + i * intervalo_tempo
            data_formatada = data_atual.isoformat() + "Z"  # Formata no padrão ISO 8601 (necessário para o TimestampedGeoJson)
            linhas.append({"coord": coords, "datas": data_formatada})
        else:
            print(f"Coordenadas não encontradas para o hop {hop}. Saltando este hop.")

    # Construir as features do mapa com as coordenadas coletadas
    features = []
    for i in range(len(linhas) - 1):  # Itera pelas linhas criando as conexões
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [linhas[i]["coord"], linhas[i+1]["coord"]]  # Conecta o ponto i com o ponto i+1
            },
            "properties": {
                "times": [linhas[i]["datas"], linhas[i + 1]["datas"]],
                "style": {"color": "#16BAB6", "weight": 3},
                "icon": "circle",
                "iconstyle": {"fillColor": "#16BAB6", "fillOpacity": 0.6, "radius": 20}
            }
        }
        features.append(feature)

    # Criar o mapa com o ponto inicial sendo o primeiro hop
    if linhas:
        inicial_latlon = linhas[0]["coord"]
        mapa = folium.Map(location=inicial_latlon, tiles="cartodbpositron", zoom_start=5)

        # Adicionar os dados de rota animados ao mapa
        TimestampedGeoJson(
            data={"type": "FeatureCollection", "features": features},
            period="PT1M",  # Tempo entre os "frames" da animação
            add_last_point=True,  # Adiciona o último ponto
            loop=False,
            auto_play=True  # Habilita o auto play
        ).add_to(mapa)
        return mapa
    else:
        print("Nenhuma coordenada válida encontrada para exibir no mapa.")
        return None



## Execução do Programa



In [42]:
def main():
    destino = input("Digite o IP ou endereço do host: ")
    try:
        hops = traceroute_with_ping(destino)
        print("hops ", hops)
        mapa = plot_route(hops)
        mapa.save("mapa.html")
    except socket.gaierror:
        print(f"Erro: Host '{destino}' não encontrado. Verifique o endereço e tente novamente.")
    except Exception as e:
        print(f"Erro inesperado: {e}")
main()

TraceRoute para google.com:
1: 172.19.32.1 | Sent: 4 | Loss: 0.0 % | avg: 0.195ms | Best: 0.169ms | Worst: 0.233ms | Std.Dev: 0.024
2: 192.168.15.1 | Sent: 4 | Loss: 0.0 % | avg: 0.856ms | Best: 0.703ms | Worst: 1.031ms | Std.Dev: 0.128
3: * (timeout)
4: 201.1.224.38 | Sent: 4 | Loss: 0.0 % | avg: 5.206ms | Best: 2.405ms | Worst: 11.981ms | Std.Dev: 3.925
5: 152.255.197.26 | Erro no ping
6: 152.255.152.15 | Erro no ping
7: 152.255.160.241 | Erro no ping
8: * (timeout)
9: 72.14.243.38 | Sent: 4 | Loss: 0.0 % | avg: 60.435ms | Best: 59.612ms | Worst: 61.011ms | Std.Dev: 0.524
10: * (timeout)
11: 216.239.42.204 | Sent: 4 | Loss: 0.0 % | avg: 60.69ms | Best: 60.635ms | Worst: 60.779ms | Std.Dev: 0.054
12: 142.251.48.153 | Sent: 4 | Loss: 0.0 % | avg: 58.966ms | Best: 58.718ms | Worst: 59.287ms | Std.Dev: 0.234
13: 192.178.99.117 | Sent: 4 | Loss: 0.0 % | avg: 59.87ms | Best: 59.353ms | Worst: 60.568ms | Std.Dev: 0.439
14: 142.250.78.238 | Sent: 4 | Loss: 0.0 % | avg: 60.222ms | Best: 59.73